一般来说，in文件内容的主要框架包含以下几个部分：
 1. 初始模拟系统设置
 2. 初始模型构建（读取模型数据）
 3. 定义原子间相互作用势或设置力场（势文件或力场文件）
 4. 定义原子/体系某些信息的计算（原子势能。。）
 5. 定义输出原子（坐标）/体系（热力学）信息
 6. 模拟环境设定并运行

In [ ]:
%%writefile relax.in
# 初始模拟系统设置
units        metal    # 单位选择十分重要
atom_style   atomic
boundary     p p p    # 定义边界条件

# 初始模型构建（读取模型数据）
lattice      bcc 3.168
region       box block 0 5 0 5 0 5
create_box   1 box
create_atoms 1 box

# 定义原子间相互作用势或设置力场（势文件或力场文件）
pair_style   eam/alloy
pair_coeff   * * W_zhou.eam.alloy W  # * * 表示考虑任意两个原子间的相互作用

# 定义原子/体系某些信息的计算（原子势能。。）
compute      1 all pe/atom

# 定义输出原子（坐标）/体系（热力学）信息
thermo       10
thermo_style custom step temp pe etotal dt time
dump         1 all custom 100 W.xyz id type x y z

# 模拟环境设定并运行
velocity     all create 300 666 mom yes rot yes dist gaussian
fix          1 all nvt temp 300 300 0.05
timestep     0.001
run          10000

In [ ]:
!"E:/LAMMPS/bin/lmp" < relax.in

# 初始模拟系统设置
### unit metal
定义整个模拟中的单位
### boundary p p p
boundary命令是用来定义边界条件的，LAMMPS提供了四种边界条件  
p：周期性边界条件  
f：非周期性边界条件，采用这种边界条件，当有原子运动到盒子以外的区域时，该原子便会被系统删除。需与thermo_modify和lost ignore 这条命令结合使用。  
s：该方向的长度会随着原子的运动而改变（以保证不丢失原子）。  
m：该方向的大小会随着原子的运动而改变，但该方向长度有最小值。  
boundary    p p p (三个方向均为周期性边界条件)  
boundary    p p f (只有z方向为f)  
boundary    p p s  
boundary    p p m  
### atom_style atomic  
告诉LAMMPS模拟的体系中有什么比如说原子，键角，电荷之类的，  
若模型中只有原子：  
atom_style       atomic（一般金属体系就选这个即可）  
若模型中还要考虑键角作用，则：  
atom_style       angle （可能适合于水分子模型模拟）  
### neighbor skin style
- skin=extra distance beyond force cutoff(distance units)
- style=bin / nsp / multi
LAMMPS在计算各个粒子间作用力时，当两个原子的距离超过截断半径时，它们之间的作用将不被考虑。但是，分子动力学程序在运行时，如果每一步都要判断每个原子的截断半径内有哪些原子，所耗费的计算资源也会随之原子数的增加而指数级增加。为了并行加速计算，LAMMPS构建邻域列表节省原子之间距离的判断的时间。
neighbor命令一般可以不用设置，即保持默认值即可，体系较大时可以适当与neighbor_modify（定义邻域列表的更新频率）修改参数减少计算量（增大bin的值以及降低更新列表的频率。。）

### dimension 
定义模拟体系维度的，一般默认为3维（所以一般不用管），也可以设置为2维

# 初始模型构建
## read_data  file.dat
直接读取模型文件
## 使用命令创建简单模型
- lattice
- region
- create_box
- create_atoms
- group
- delete_atoms

### lattice
定义晶格类型，晶格常数，以及晶向方向  
lattice bcc 3.168 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1  
构建一个晶格常数为3.168，晶格类型为bcc的模型，其x方向的矢量为 [100]。。。。。  
默认值  
orient x 1 0 0 orienty 0 1 0 orient z 0 0 1  
### region   
构建模拟盒子大小以及划分模拟区域  
region box block 0 10 0 10 0 10 units lattice  
表示构建一个10a*10a*10a大小的模拟盒子（a为晶格常数），  
units lattice为默认值  
可以换成units box[那么这个盒子的大小就是10*10*10（埃）]  
region 1 block 1 9 1 9 1 9  
表示将盒子中x（1a-9a），y（1a-9a），z（1a-9a）的区域选中，定义为区域1，用于后续模拟。  
注：一般box这个单词只用于与盒子相关的地方，只选取部分区域时，不要将其定义为box。  

### create_box  
LAMMPS模型中有几类原子，几类键等等  
create_box 2 box  
表示盒子中有2种原子。  
create_box 2 box bond/type 2  
表示盒子中有2种原子以及2种键长。  

### create_atoms
表示往盒子中添加原子  
create_atoms 1 box  
表示将类型1的原子按照lattice命令的设定填满盒子。  
create_atoms 2 single 5 5 5  
表示在坐标为（5a, 5a, 5a）的地方添加一个类型为2的原子。  

### group 
表示将选中的原子定义为一个组，用于后续模拟    
group 1 type 1  
表示将所有类型为1的原子设置为1组；  
group 2 region 2  
表示将处于区域2中的所有原子设置为2组；  
group 3 union 1 2  
表示将1，2组合并为3组。  

### delete_atoms命令：
表示删除不需要的原子，例如  
delete_atoms group1  
表示将1组的原子删掉；  
delete_atoms region 2  
表示将区域2中的原子删掉；  

### 结构优化 静态优化方法 分子静力学方法
`minimize 1.0e-12 1.0e-12 10000 10000`    
1.0e-12:力的收敛精度  
1.0e-12:能量的收敛精度  
10000:最大迭代次数  
10000:力和能量的最大评估次数  

### 定义原子/体系信息计算
计算每个原子势能 1为compute代号  
`compute 1 all pe/atom`   
计算每个原子在x,y,z方向受力  
`compute 1 all property/atom fx fy fz`     
在命令`dump 1 all custom 100 W.xyz id type x y z`后面加上c_1(表示输出计算的原子势能)：    
`dump 1 all custom 100 W.xyz id type x y z c_1`    
c为compute的缩写，1为这个compute的代号，    
若写为`compute x all pe/atom`, 则输出的写为c_x  

### 定义输出原子/体系(热力学)信息(1)
`thermo` 定义多少步输出一次热力学信息  
`thermo_style` 输出需要的热力学信息  
`thermo_tyle style args`  
`thermo_style custom step temp ke pe etotal`  
其中`custom`表示所需的热力学信息由我们自定义：  
- `step`表示输出运行的步数是多少   
- `temp`表示体系的温度  
- `ke`表示体系的动能  
- `pe`表示体系的势能  
- `pxx,pyy,pzz,pxy...`表示体系各个方向的压强  

### 定义输出原子/体系(热力学)信息(2)
一般重要结果都是通过`dump`输出    
`dump 1 all custom 500 file_name.xyz id type x y z`    
`all`表示对于所有原子（当然也有只输出一部分原子的表示方式，与之前介绍的group命令相关），500表示每运行500步输出一次`file_name.xyz`为输出文件的文件名，而其中的内容有：
- `id`：原子的序号
- `type`：原子的类型
- `x,y以及z`：原子的坐标

### 模拟环境设定
`velocity` `fix` `timestep` `run`  
#### velocity
分子动力学模拟，模拟过程中原子是动的，模拟初期我们需要给与原子初速度。  
`velocity all create 300.0 4928459 dist gaussian`    
其中`all`表示赋予所有原子   
`300`为300 K；  
`4928459`为随机正整数；   
`dist gaussian`为原子速度分布满足高斯分布。   
#### timestep
设置模拟步长
`timestep 0.001` (ps还是fs与之前介绍的units命令有关)     
#### fix   
设置模拟系综（nve,nvt,npt等）  
`fix 1 all nvt temp 300 300 100.0`  
控制体系在温度为300K左右的nvt环境下模拟
对原子进行操作  
`fix 2 bottom setforce 0.0 0.0 0.0`  
设定bottom区域（region命令）的原子的受力为0（固定原子时会用到）   
帮助输出所需信息，例：
`fix extra all print 100 "Coords of marker atom =$x $y $z" file coord.txt`
#### run
设置运行步数
`run 10000`
表示运行10000步，运行步数与步长相乘即可得模拟时间（不是计算时间~）